<a href="https://colab.research.google.com/github/Haswanthobbina/mri-images-classification/blob/main/notebookphase2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
masoudnickparvar_brain_tumor_mri_dataset_path = kagglehub.dataset_download('masoudnickparvar/brain-tumor-mri-dataset')

print('Data source import complete.')


100%|██████████| 157M/157M [00:02<00:00, 81.1MB/s]

Extracting files...


Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:

# =================================================================
# 🚀 ULTIMATE MASTER SCRIPT: 5 DL Models + 3 Hybrid ML Models
# =================================================================
# FEATURES:
#  1. Trains ResNet18, VGG16, MobileNetV2, EfficientNet, DenseNet.
#  2. Extracts Deep Features (DenseNet) + Handcrafted Features (GLCM, Symmetry).
#  3. Trains Random Forest, XGBoost, SVM on fused features.
#  4. Generates a 'final_project_bundle.zip' with all models & metrics.

# 1. INSTALL DEPENDENCIES
!pip install -q kagglehub scikit-learn pandas xgboost scikit-image opencv-python

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import kagglehub
import os
import json
import shutil
import numpy as np
import pandas as pd
import joblib
import cv2
from PIL import Image
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
from skimage.feature import graycomatrix, graycoprops
from scipy.stats import skew, kurtosis

# ---------------------------------------------------------
# 2. SETUP & DATA DOWNLOAD
# ---------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Running on: {device}")

print("⬇️ Downloading dataset...")
path = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")
TRAIN_DIR = os.path.join(path, "Training")
TEST_DIR = os.path.join(path, "Testing")

# Define Output Directory
OUTPUT_DIR = "/content/final_models_output"
if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)
os.makedirs(OUTPUT_DIR)

# Data Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_data = datasets.ImageFolder(TRAIN_DIR, transform=transform)
test_data = datasets.ImageFolder(TEST_DIR, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)

# ---------------------------------------------------------
# 3. DEFINE DEEP LEARNING MODELS
# ---------------------------------------------------------
def get_model(model_name, pretrained=True):
    weights = 'DEFAULT' if pretrained else None

    if model_name == "ResNet-18":
        model = models.resnet18(weights=weights)
        model.fc = nn.Linear(model.fc.in_features, 4)

    elif model_name == "VGG-16":
        model = models.vgg16(weights=weights)
        model.classifier[6] = nn.Linear(4096, 4)

    elif model_name == "MobileNetV2":
        model = models.mobilenet_v2(weights=weights)
        model.classifier[1] = nn.Linear(model.last_channel, 4)

    elif model_name == "EfficientNet-B0":
        model = models.efficientnet_b0(weights=weights)
        model.classifier[1] = nn.Linear(1280, 4)

    elif model_name == "DenseNet-121":
        model = models.densenet121(weights=weights)
        model.classifier = nn.Linear(1024, 4)

    return model.to(device)

# ---------------------------------------------------------
# 4. TRAIN & EVALUATE 5 DL MODELS
# ---------------------------------------------------------
dl_models = ["ResNet-18", "VGG-16", "MobileNetV2", "EfficientNet-B0", "DenseNet-121"]
metrics_results = {}

print("\n🔥 STARTING DEEP LEARNING TRAINING...")

for name in dl_models:
    print(f"\n➡️ Processing {name}...")
    model = get_model(name)
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    criterion = nn.CrossEntropyLoss()

    # Train (3 Epochs)
    model.train()
    for epoch in range(5):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"   Epoch {epoch+1}: Loss {running_loss/len(train_loader):.4f}")

    # Evaluate
    model.eval()
    all_preds, all_labels, all_probs = [], [], []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            probs = F.softmax(outputs, dim=1)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    # Calculate Metrics
    report = classification_report(all_labels, all_preds, output_dict=True, zero_division=0)
    try:
        auc = roc_auc_score(all_labels, all_probs, multi_class='ovr', average='macro')
    except:
        auc = 0.0

    safe_name = name.lower().replace("-", "").replace("_", "")
    metrics_results[safe_name] = {
        "accuracy": accuracy_score(all_labels, all_preds),
        "precision": report['macro avg']['precision'],
        "recall": report['macro avg']['recall'],
        "f1_score": report['macro avg']['f1-score'],
        "auc_roc": auc,
        "description": f"{name} Deep Learning Model"
    }

    # SAVE MODEL
    save_path = os.path.join(OUTPUT_DIR, f"{safe_name}_brain_tumor.pth")
    torch.save(model.state_dict(), save_path)
    print(f"   ✅ Saved {name} to {save_path}")

# ---------------------------------------------------------
# 5. FEATURE ENGINEERING (Hybrid Extraction)
# ---------------------------------------------------------
print("\n🧬 STARTING HYBRID FEATURE EXTRACTION...")

# A. Handcrafted Features Function
def get_handcrafted_features(image_path):
    try:
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is None: return np.zeros(11) # Fallback
        img = cv2.resize(img, (224, 224))

        # Stats
        mean_val = np.mean(img)
        std_val = np.std(img)
        var_val = np.var(img)
        skew_val = skew(img.flatten())
        kurt_val = kurtosis(img.flatten())

        # Texture (GLCM)
        glcm = graycomatrix(img, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
        contrast = graycoprops(glcm, 'contrast').mean()
        dissimilarity = graycoprops(glcm, 'dissimilarity').mean()
        homogeneity = graycoprops(glcm, 'homogeneity').mean()
        energy = graycoprops(glcm, 'energy').mean()
        correlation = graycoprops(glcm, 'correlation').mean()

        # Symmetry
        h, w = img.shape
        left = img[:, :w//2]
        right = img[:, w//2:]
        right_flip = cv2.flip(right, 1)
        right_flip = cv2.resize(right_flip, (left.shape[1], left.shape[0]))
        asymmetry = np.mean(np.abs(left - right_flip))

        return np.array([mean_val, std_val, var_val, skew_val, kurt_val,
                         contrast, dissimilarity, homogeneity, energy, correlation, asymmetry])
    except:
        return np.zeros(11)

# B. Load Deep Feature Extractor (DenseNet-121)
feature_extractor = get_model("DenseNet-121", pretrained=False)
densenet_path = os.path.join(OUTPUT_DIR, "densenet121_brain_tumor.pth")
feature_extractor.load_state_dict(torch.load(densenet_path))
feature_extractor.classifier = nn.Identity()
feature_extractor.eval()

# C. Extraction Loop
def extract_hybrid_data(dataset):
    features_list, labels_list = [], []

    for i in range(len(dataset)):
        path, label = dataset.samples[i]

        # 1. Deep Features
        img_pil = Image.open(path).convert("RGB")
        tensor = transform(img_pil).unsqueeze(0).to(device)
        with torch.no_grad():
            deep_feat = feature_extractor(tensor).cpu().numpy().flatten()

        # 2. Handcrafted Features
        manual_feat = get_handcrafted_features(path)

        # 3. Fuse
        combined = np.hstack([deep_feat, manual_feat])

        features_list.append(combined)
        labels_list.append(label)

        if i % 500 == 0: print(f"   Processed {i}/{len(dataset)}...", end='\r')

    return np.vstack(features_list), np.array(labels_list)

print("   Extracting Training Features (This takes a moment)...")
X_train, y_train = extract_hybrid_data(train_data)
print(f"\n   Training Features Shape: {X_train.shape}")

print("   Extracting Testing Features...")
X_test, y_test = extract_hybrid_data(test_data)

# ---------------------------------------------------------
# 6. TRAIN ML MODELS (RF, XGB, SVM)
# ---------------------------------------------------------
ml_models_map = {
    "RandomForest": RandomForestClassifier(n_estimators=100),
    "XGBoost": xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'),
    "SVM": SVC(probability=True)
}

print("\n🤖 TRAINING MACHINE LEARNING MODELS...")
for name, clf in ml_models_map.items():
    print(f"➡️ Training {name}...")
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    y_prob = clf.predict_proba(X_test)

    # Metrics
    report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
    try:
        auc = roc_auc_score(y_test, y_prob, multi_class='ovr', average='macro')
    except:
        auc = 0.0

    metric_key = f"ml_{name.lower()}"
    metrics_results[metric_key] = {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": report['macro avg']['precision'],
        "recall": report['macro avg']['recall'],
        "f1_score": report['macro avg']['f1-score'],
        "auc_roc": auc,
        "description": f"Hybrid Model: DenseNet + Handcrafted + {name}"
    }

    joblib.dump(clf, os.path.join(OUTPUT_DIR, f"{metric_key}.pkl"))
    print(f"   ✅ Saved {name}")

# ---------------------------------------------------------
# 7. SAVE METRICS & ZIP
# ---------------------------------------------------------
print("\n📦 ZIPPING FINAL RESULTS...")
with open(os.path.join(OUTPUT_DIR, "model_metrics.json"), "w") as f:
    json.dump(metrics_results, f, indent=4)

shutil.make_archive("/content/final_project_bundle", 'zip', OUTPUT_DIR)

from google.colab import files
print("🎉 SUCCESS! Downloading 'final_project_bundle.zip'...")
files.download('/content/final_project_bundle.zip')

🚀 Running on: cuda
⬇️ Downloading dataset...
Using Colab cache for faster access to the 'brain-tumor-mri-dataset' dataset.

🔥 STARTING DEEP LEARNING TRAINING...

➡️ Processing ResNet-18...
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 189MB/s]


   Epoch 1: Loss 0.2309
   Epoch 2: Loss 0.0416
   Epoch 3: Loss 0.0222
   Epoch 4: Loss 0.0214
   Epoch 5: Loss 0.0079
   ✅ Saved ResNet-18 to /content/final_models_output/resnet18_brain_tumor.pth

➡️ Processing VGG-16...
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


100%|██████████| 528M/528M [00:06<00:00, 82.0MB/s]


   Epoch 1: Loss 0.3635
   Epoch 2: Loss 0.1159
   Epoch 3: Loss 0.0770
   Epoch 4: Loss 0.0425
   Epoch 5: Loss 0.0429
   ✅ Saved VGG-16 to /content/final_models_output/vgg16_brain_tumor.pth

➡️ Processing MobileNetV2...
Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-7ebf99e0.pth


...

   Epoch 1: Loss 0.2466
   Epoch 2: Loss 0.0481
   Epoch 3: Loss 0.0249
   Epoch 4: Loss 0.0207
   Epoch 5: Loss 0.0193
   ✅ Saved DenseNet-121 to /content/final_models_output/densenet121_brain_tumor.pth

🧬 STARTING HYBRID FEATURE EXTRACTION...
   Extracting Training Features (This takes a moment)...

   Training Features Shape: (5600, 1035)
   Extracting Testing Features...

🤖 TRAINING MACHINE LEARNING MODELS...
➡️ Training RandomForest...
   ✅ Saved RandomForest
➡️ Training XGBoost...


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [10:52:08] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


   ✅ Saved XGBoost
➡️ Training SVM...
   ✅ Saved SVM

📦 ZIPPING FINAL RESULTS...
🎉 SUCCESS! Downloading 'final_project_bundle.zip'...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>